In [ ]:
import os
import shutil
import opendatasets as od
import pandas as pd
from IPython.display import display
from database.sql_connector import SqlConnector

In [ ]:
sql_con = SqlConnector()

## Netflix Originals

In [ ]:
kaggle_user = "dikshabhati2002"
folder_name = "walt-disney-movies"

In [ ]:
od.download(f"https://www.kaggle.com/{kaggle_user}/{folder_name}", force=True)

In [ ]:
file = os.listdir(folder_name)[0]
df = pd.read_csv(os.path.join(folder_name,file), encoding="ISO-8859-1")
df = df[['title', 'Release date', 'Running time', 'Budget', 'Box office',
         'imdb', 'metascore', 'rotten_tomatoes']]

### Pre-process release date column 

In [ ]:
df['Release date'] = pd.to_datetime(df['Release date'], format="%Y-%m-%d")

### Pre-process ratings column 

In [ ]:
df.rotten_tomatoes = df.rotten_tomatoes.replace('%','', regex=True).astype('float')
df.imdb = (df.imdb / 10) * 100

### Ingest data 

In [ ]:
df.rename(columns={'title':'Title',
                   'Running time':'Running_Time',
                   'Box office':'Box_Office',
                   'Release date':'Release_Date',
                   'imdb':'IMDB',
                   'metascore':'Metascore',
                   'rotten_tomatoes':'Rotten_Tomatoes'
                  }, inplace=True)

sql_con.run_query("""DELETE FROM KAGGLE.DISNEY.DATA""")
sql_con.ingest_dataframe(df, schema='DISNEY', table='DATA')
shutil.rmtree(folder_name)